## Imports

In [29]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium

## Read-in

In [35]:
df = pd.read_csv('Damaghi Family Purchases in South Florida - Sheet1.csv')

In [36]:
df = df.rename(columns={'Date and Year of Deal':'deal_date','Land size (f)':'land_size_sf',
                  'Building size, use and year of construction ':'build_size_use_year_built',
                  'Entity used to make the purchase':'buying_entity'})

In [37]:
df['Price'] = df['Price'].str.replace(',','')
df = df.fillna('Not available')
df['Location'] = df['Location'].str.replace('\n','')
df['Location'] = df['Location'].astype(str)
df['buying_entity'] = df['buying_entity'].str.replace('\n','')

## Geocode

In [38]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [40]:
df = df.drop(labels=21, axis='index')

In [42]:
def geocode(add):
    g = gmaps_key.geocode(add)
    if not g:
        # handle empty list
        return (None, None)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)


df['geocoded'] = df['Location'].apply(geocode)

In [45]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## HTML Popup Formatter

In [47]:
df.columns

Index(['deal_date', 'Property type', 'land_size_sf',
       'build_size_use_year_built', 'Price', 'Location', 'buying_entity',
       'geocoded', 'lat', 'lon'],
      dtype='object')

In [48]:
def popup_html(row):
    deal_date = row['deal_date']
    property_type = row['Property type']
    land_size_sf = row['land_size_sf']
    build_size_use_year_built = row['build_size_use_year_built']
    Price = row['Price']
    Location = row['Location']
    buying_entity = row['buying_entity']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Deal date: </strong>{}'''.format(deal_date) + '''<br>
    <strong>Property type: </strong>{}'''.format(property_type) + '''<br>
    <strong>Land size (sf): </strong>{}'''.format(land_size_sf) + '''<br>
    <strong>build_size_use_year_built: </strong>{}'''.format(build_size_use_year_built) + '''<br>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Location: </strong>{}'''.format(Location) + '''<br>
    <strong>Buying entity: </strong>{}'''.format(buying_entity) + '''<br>
    </html>
    '''
    return html

## Make map

In [58]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Damaghi Family Purchases in South Florida')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

for index, row in df.iterrows():
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(m)


folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# 
m.get_root().html.add_child(folium.Element(title_html))
            
# Display map
m

In [59]:
m.save('index.html')